Training
===
> Training of Prediction Models for Anomaly Detection


In [ ]:
#| default_exp anomaly.training

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.basics import *
from pympp.process import *
from pympp.anomaly.detect import *
from fastai.callback.all import *
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
from fastai.tabular.model import get_emb_sz
import fire

Training Procedure
--

In [ ]:
#| export
def train(fn,log_name,store_path='models',epoch=25,ws=5): 
    cols= get_attr(attr_dict,log_name)
    log = import_log(fn,cols)
    o,dls,categorify = training_dl(log,cols,ws=ws)
    p = f'{store_path}/{log_name}_vocab.p'
    with open(p, "wb") as output_file:
        pickle.dump(categorify, output_file)
    emb_szs = get_emb_sz(o)
    m=MultivariateModel(emb_szs)
    loss=partial(multi_loss_sum,o)
    train_val = train_validate(dls,m,loss=loss,metrics=get_metrics(o),epoch=epoch,show_plot=False,print_output=False,store_path=store_path,model_name=log_name)


PDC 2020
--

In [ ]:
#| export
def train_pdc20_logs(epoch=25):
    store_path='models/pdc2020'
    for training_log in progress_bar(glob.glob('data/logs/csv/dapnn_ds/PDC2020_training/*')):
        log_name = Path(training_log).stem[:-4]
        train(training_log,log_name,store_path=store_path,epoch=epoch)

In [ ]:
# train_pdc20_logs(epoch=1)

PDC 2021
--

In [ ]:
#| export
def train_pdc21_logs(epoch=25,rel_path="./"):
    store_path='models/pdc2021'
    for training_log in progress_bar(glob.glob('data/logs/csv/dapnn_ds/PDC2021_training/*')):
        log_name = Path(training_log).stem[:-4]
        train(training_log,log_name,store_path=store_path,epoch=epoch)

In [ ]:
# train_pdc21_logs(epoch=1)

Binet datasets
--

In [ ]:
#| export
def train_binet_logs(epoch=25):
    store_path='models/binet_logs'
    for training_log in progress_bar(glob.glob('data/logs/csv/dapnn_ds/binet_logs/*')):
        log_name = training_log.split('/')[-1][:-7]
        train(training_log,log_name,store_path=store_path,epoch=epoch)


In [ ]:
# train_binet_logs(epoch=1)

Shell Utils
--

In [ ]:
#| export
def run_training(log="all",epoch=25):
    if log == 'binet':
        train_binet_logs(epoch=epoch)
    elif log == 'pdc20':
        train_pdc20_logs(epoch=epoch)
    elif log == 'pdc21':
        train_pdc21_logs(epoch=epoch)
    elif log == 'all':
        
        train_pdc20_logs(epoch=epoch)
        print("PDC 2020 completed")
        train_pdc21_logs(epoch=epoch)
        print("PDC 2021 completed")
        train_binet_logs(epoch=epoch)
        print("BINET Logs completed")
    else: 
        raise ValueError(f'{log} is not a supported data set!')

In [ ]:
#| hide, |eval: false

# Runs the complete Training - takes arround 3 days
%time  run_training()

KeyboardInterrupt: 

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()